<h2>For calibration basins, combines separate rainfall and ET files into one file to be used by model calibration routines.</h2>

In [ ]:
%pylab notebook
import datetime as dt
import matplotlib.pyplot as plt
from charistools.timeSeries import TimeSeries
import pandas as pd

In [ ]:
#drainageids = ['AM_Vakhsh_at_Komsomolabad',
#               'IN_Hunza_at_DainyorBridge',
#               'GA_Karnali_at_Benighat',
#               'GA_Narayani_at_Devghat',
#               'GA_SaptaKosi_at_Chatara']
drainageids = ['SY_Naryn_at_NarynTown']

In [ ]:
fig, ax = plt.subplots(len(drainageids),1, figsize=(8,10))

In [ ]:
def orig_rainfile(source, id):
    if 'MERRA' == source:
        return "/Users/brodzik/projects/CHARIS/rainfall/MERRA/%s.MERRA_rainfall_km3.monthly.csv" % id
    else:
        return "/Users/brodzik/projects/CHARIS/rainfall/Aphrodite_V1101R2/monthly_by_drainage/%s.APHRODITE_rainfall_km3.monthly.v01.csv" % id

def new_rainfile(source, id):
    if 'MERRA' == source:
        return "/Users/brodzik/projects/CHARIS/rainfall/MERRA_rainfall_less_ET/%s.MERRA_rainfall_adj_mod16_ET_km3.monthly.csv" % id
    else:
        return "/Users/brodzik/projects/CHARIS/rainfall/Aphrodite_V1101R2/monthly_by_drainage_less_et/%s.APHRODITE_rainfall_adj_mod16_ET_km3.monthly.v01.csv" % id
    

In [ ]:
for source in ['MERRA', 'Aphrodite']:
    print(source)
    print(orig_rainfile(source, drainageids[0]))
    print(new_rainfile(source, drainageids[0]))

In [ ]:
for i, id in enumerate(drainageids):
    source = "Aphrodite"
    rainfallFile = orig_rainfile(source, id)
    ETFile = '/Users/brodzik/projects/CHARIS/evapotranspiration/mod16/' + id + '.mod16.ET.csv'
    rainfall = TimeSeries(rainfallFile)
    et = TimeSeries(ETFile)
    rainfall.data['et_km3'] = et.data["et"]
    rainfall.data['diff_km3'] = rainfall.data["rainfall"] - rainfall.data["et_km3"]
    rainfall.data['rainfall-et_km3'] = rainfall.data['diff_km3']
    rainfall.data.loc[rainfall.data['rainfall-et_km3'] < 0., 'rainfall-et_km3'] = 0.
    #print(rainfall.data)
    
    if len(drainageids) > 1:
        ax[i].plot(rainfall.data['rainfall'], color='b', label="%s rainfall" % source)
        ax[i].plot(rainfall.data['et_km3'], color='g', label='ET')
        ax[i].plot(rainfall.data['rainfall-et_km3'], color='r', label="%s rainfall-ET" % source)
        ax[i].set_title(id)
        ax[i].legend()
    else:
        ax.plot(rainfall.data['rainfall'], color='b', label="%s rainfall" % source)
        ax.plot(rainfall.data['et_km3'], color='g', label='ET')
        ax.plot(rainfall.data['rainfall-et_km3'], color='r', label="%s rainfall-ET > 0" % source)
        ax.plot(rainfall.data['diff_km3'], color='y', label="%s rainfall-ET" % source)
        ax.set_title(id)
        ax.legend()
    newFile = new_rainfile(source, id)
    rainfall.comments.insert(-2,"# Added ET by MJB on %s" % str(dt.datetime.now()))
    rainfall.comments.insert(-2,"# ET from : %s" % ETFile)
    rainfall.write(newFile)
plt.tight_layout()        

In [ ]:
plt.savefig('/Users/brodzik/projects/CHARIS/rainfall/Aphrodite_V1101R2/%s.Aphrodite.rainfall-ET.png' % drainageids[0])

In [ ]:
%pwd

In [ ]:
%cd /Users/brodzik/projects/CHARIS/rainfall/MERRA
%ls

In [ ]:
file = 'AM_Vakhsh_at_Komsomolabad.MERRA_rainfall_km3.monthly.csv'
data = TimeSeries(file)
data.data

In [ ]:
fig, ax = plt.subplots(len(drainageids),1, figsize=(8,10))
for i, id in enumerate(drainageids):
    merraRainfallFile = new_rainfile('MERRA', id)
    aphRainfallFile = new_rainfile('Aphrodite', id)
    merraRainfall = TimeSeries(merraRainfallFile)
    aphRainfall = TimeSeries(aphRainfallFile)
    
    ax[i].plot(aphRainfall.data['rainfall-et_km3'], color='r', 
               label='Aphrodite rainfall-ET')
    ax[i].plot(merraRainfall.data['rainfall-et_km3'], color='k', 
               label='MERRA rainfall-ET')
    
    ax[i].set_title(id)
    ax[i].legend()
plt.tight_layout()

In [ ]:
plt.savefig('/Users/brodzik/projects/CHARIS/rainfall/MERRA_rainfall_less_ET/calibration_basins.Aphrodite.v01_vs_MERRA.rainfall-ET.png')